**Задание 1: Векторизация текста с использованием Мешка слов (BoW) и TF-IDF**

1. Создайте корпус из минимум 10 текстовых документов (можно взять новостные статьи, отзывы на товары, статьи по одной тематике).
2. Реализуйте предобработку текста, включающую:
* Приведение к нижнему регистру
* Токенизацию
* Удаление стоп-слов и пунктуации
* Лемматизацию/стемминг
3. Реализуйте модель Мешка слов (BoW) с использованием CountVectorizer из scikit-learn.
4. Реализуйте модель TF-IDF с использованием TfidfVectorizer из scikit-learn.
5. Найдите 10 самых значимых терминов для каждого документа по обоим подходам и сравните результаты.
6. Визуализируйте сходство документов с помощью метрики косинусного расстояния для обоих подходов.
7. Прокомментируйте разницу в результатах между BoW и TF-IDF

In [6]:
import re
import string
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# ваш код
documents = [
    "В целом и общем, впечатления от использования приятные: телефон не тупит, сенсор не глючит, батарея, обычная уже собственно для бюджетного класса, держит вполне прилично - при средней активности использования (постоянно мессенджеры и периодически видео) ушло за сутки в аккурат 50%.",
    "Постоянно лагает и тормозит. Для нового телефона - это катастрофа! Нечувствительный тачскрин. Фото с близкого расстояния - нечеткие. Всплывающие окна с рекламой - просто каждую секунду, не в каждом приложении ее можно заблокировать.",
    "Хороший, шустрый аппарат, зарядки при активном использовании хватает на сутки.",
    "Я считаю это лучший телефон за свои деньги. Из главных особенностей Amoled и мини Джек. Рекомендую к покупке. За свои деньги отлично!",
    "За эту цену отличный смартфон. Если не нужно сейчас, то можно на 7 тысяч с али подождать. Все на уровне. Рабочая лошадка.",
    "Не тормозит, Камера норм, выглядит стильно. За свою цену норм, сейчас цена стала неадекватной.",
    "За свои деньги хороший аппарат. Работает шустро, не тормозит. В комплекте есть чехол. Нет широкоугольной камеры. Чехол из комплекта не защищает камеру. На экране уже наклеена защитная плёнка. Вторую симку нельзя вставить, если используетсяSD карта.",
    "Поною из категории 'раньше было лучше' - раньше новый телефон (даже в такой же ценовой категории) был реально чем-то новым, на порядок лучшая камера, новые функции и т.д., ощущалась некая радость даже от покупки. Сейчас - только досада от затраты времени на перелогин во всех приложениях, но аккумуляторы и всё остальное не вечны, менять приходится.",
    "Пользовался сяомами с момента их появления и всегда это было топ за свои деньги и даже лучше. Вот решил обновить свой четырехлетний редми ноут 11 на сей агрегат. Но 14 конкретно подкачал, я так понимаю главная беда это хайпер ос, куча неубираемого мусора, отсутствие нужных настроек бывших в старших версиях, камера по моему еще хуже чем в 11.",
    "Взял в феврале 25 после самсунга А51, которым пользовался пять лет (начала сдыхать батарея). Пришел за Tecno, но продавец в магазине начал садиться на уши и рекомендовать сие произведение искусства. Из плюсов: батарея действительно крепкая, а динамик на максимуме очень громкий. Встроенная запись разговоров может кому-нибудь пригодится. Из минусов: отвратительная лагучая ось, постоянно выкидывающая какие-либо приколы. Особенно после обновлений. Тупящий датчик освещения, который может в случайный момент (особенно он любит это делать вечером перед сном в полной темноте) включить яркость на максимум и ослепить тебя на полминуты."
]
nltk.download('stopwords')
nltk.download('punkt_tab')

russian_stopwords = stopwords.words('russian')
stemmer = SnowballStemmer('russian')

def preprocess_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = re.sub(r'\d+', '', text)
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in russian_stopwords and len(t) > 2]
    stemmed = [stemmer.stem(t) for t in tokens]
    return " ".join(stemmed)

preprocessed_documents = [preprocess_text(doc) for doc in documents]

print("Предобработанные отзывы:")
for i, doc in enumerate(preprocessed_documents, 1):
    print(f"{i}: {doc}")

Предобработанные отзывы:
1: цел общ впечатлен использован приятн телефон туп сенсор глюч батаре обычн собствен бюджетн класс держ вполн приличн средн активн использован постоя мессенджер периодическ виде ушл сутк аккурат
2: постоя лага тормоз нов телефон эт катастроф нечувствительн тачскрин фот близк расстоян нечетк всплыва окн реклам прост кажд секунд кажд приложен заблокирова
3: хорош шустр аппарат зарядк активн использован хвата сутк
4: счита эт лучш телефон сво деньг главн особен amoled мин джек рекоменд покупк сво деньг отличн
5: цен отличн смартфон нужн тысяч ал подожда уровн рабоч лошадк
6: тормоз камер норм выгляд стильн цен норм цен стал неадекватн
7: сво деньг хорош аппарат работа шустр тормоз комплект чехол широкоугольн камер чехол комплект защища камер экран накле защитн пленк втор симк встав используетсясд карт
8: пон категор раньш раньш нов телефон ценов категор реальн чемт нов порядок лучш камер нов функц ощуща нек радост покупк досад затрат времен перелогин приложен акк

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
count_vectorizer = CountVectorizer()
bow_matrix = count_vectorizer.fit_transform(preprocessed_documents)
bow_feature_names = count_vectorizer.get_feature_names_out()

bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=bow_feature_names,
    index=[f"Отзыв {i}" for i in range(1, 11)]
)

print("\nМатрица Bag of Words:\n")
print(bow_df)


Матрица Bag of Words:

          amoled  tecno  агрегат  аккумулятор  аккурат  активн  ал  аппарат  \
Отзыв 1        0      0        0            0        1       1   0        0   
Отзыв 2        0      0        0            0        0       0   0        0   
Отзыв 3        0      0        0            0        0       1   0        1   
Отзыв 4        1      0        0            0        0       0   0        0   
Отзыв 5        0      0        0            0        0       0   1        0   
Отзыв 6        0      0        0            0        0       0   0        0   
Отзыв 7        0      0        0            0        0       0   0        1   
Отзыв 8        0      0        0            1        0       0   0        0   
Отзыв 9        0      0        1            0        0       0   0        0   
Отзыв 10       0      1        0            0        0       0   0        0   

          батаре  бед  ...  цен  ценов  чемт  четырехлетн  чехол  \
Отзыв 1        1    0  ...    0      0

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_feature_names,
    index=[f"Отзыв {i}" for i in range(1, 11)]
)
print("\nМатрица TF-IDF: \n")
print(tfidf_df)



Матрица TF-IDF: 

            amoled     tecno   агрегат  аккумулятор   аккурат    активн  \
Отзыв 1   0.000000  0.000000  0.000000     0.000000  0.195932  0.166560   
Отзыв 2   0.000000  0.000000  0.000000     0.000000  0.000000  0.000000   
Отзыв 3   0.000000  0.000000  0.000000     0.000000  0.000000  0.337723   
Отзыв 4   0.268098  0.000000  0.000000     0.000000  0.000000  0.000000   
Отзыв 5   0.000000  0.000000  0.000000     0.000000  0.000000  0.000000   
Отзыв 6   0.000000  0.000000  0.000000     0.000000  0.000000  0.000000   
Отзыв 7   0.000000  0.000000  0.000000     0.000000  0.000000  0.000000   
Отзыв 8   0.000000  0.000000  0.000000     0.165416  0.000000  0.000000   
Отзыв 9   0.000000  0.000000  0.174495     0.000000  0.000000  0.000000   
Отзыв 10  0.000000  0.121075  0.000000     0.000000  0.000000  0.000000   

                ал   аппарат    батаре       бед  ...       цен     ценов  \
Отзыв 1   0.000000  0.000000  0.166560  0.000000  ...  0.000000  0.000000   
О

In [14]:
def top_terms_per_doc(matrix, feature_names, top_n=10, is_tfidf=False):
    tops = {}
    for i in range(matrix.shape[0]):
        row = matrix[i].toarray().flatten()
        idx_sorted = np.argsort(row)[::-1]
        top_idx = idx_sorted[:top_n]
        if is_tfidf:
            tops[i] = [(feature_names[j], round(row[j], 4)) for j in top_idx if row[j] > 0]
        else:
            tops[i] = [(feature_names[j], int(row[j])) for j in top_idx if row[j] > 0]
    return tops

bow_top_per_doc = top_terms_per_doc(bow_matrix, bow_feature_names)
tfidf_top_per_doc = top_terms_per_doc(tfidf_matrix, tfidf_feature_names, is_tfidf=True)

print("\nТоп-10 BoW по отзывам:")
for i, terms in bow_top_per_doc.items():
    print(f"\nОтзыв {i+1}:")
    for w, v in terms:
        print("  -", w, v)

print("\nТоп-10 TF-IDF по отзывам:")
for i, terms in tfidf_top_per_doc.items():
    print(f"\nОтзыв {i+1}:")
    for w, v in terms:
        print("  -", w, v)

bow_cosine = cosine_similarity(bow_matrix)
tfidf_cosine = cosine_similarity(tfidf_matrix)

print("\nКосинусное сходство BoW:")
print(pd.DataFrame(bow_cosine.round(3)))

print("\nКосинусное сходство TF-IDF:")
print(pd.DataFrame(tfidf_cosine.round(3)))

bow_sum = np.sum(bow_matrix.toarray(), axis=0)
tfidf_sum = np.sum(tfidf_matrix.toarray(), axis=0)

bow_top_corpus = sorted(zip(bow_feature_names, bow_sum), key=lambda x: x[1], reverse=True)[:10]
tfidf_top_corpus = sorted(zip(tfidf_feature_names, tfidf_sum), key=lambda x: x[1], reverse=True)[:10]

print("\nТОП-10 BoW по корпусу:")
for i, (w, f) in enumerate(bow_top_corpus, 1):
    print(f"{i:2d}. {w:<15} {f}")

print("\nТОП-10 TF-IDF по корпусу:")
for i, (w, s) in enumerate(tfidf_top_corpus, 1):
    print(f"{i:2d}. {w:<15} {s:.4f}")

bow_words = {w for w, _ in bow_top_corpus}
tfidf_words = {w for w, _ in tfidf_top_corpus}
common = bow_words & tfidf_words
bow_only = bow_words - tfidf_words
tfidf_only = tfidf_words - bow_words

print("\nОбщие слова (BoW и TF-IDF):", ", ".join(sorted(common)))
print("Только BoW:", ", ".join(sorted(bow_only)))
print("Только TF-IDF:", ", ".join(sorted(tfidf_only)))


Топ-10 BoW по отзывам:

Отзыв 1:
  - использован 2
  - цел 1
  - туп 1
  - телефон 1
  - средн 1
  - ушл 1
  - сенсор 1
  - собствен 1
  - мессенджер 1
  - общ 1

Отзыв 2:
  - кажд 2
  - эт 1
  - тормоз 1
  - телефон 1
  - тачскрин 1
  - фот 1
  - реклам 1
  - расстоян 1
  - нов 1
  - нечувствительн 1

Отзыв 3:
  - шустр 1
  - хорош 1
  - хвата 1
  - сутк 1
  - активн 1
  - аппарат 1
  - зарядк 1
  - использован 1

Отзыв 4:
  - сво 2
  - деньг 2
  - счита 1
  - рекоменд 1
  - телефон 1
  - эт 1
  - лучш 1
  - джек 1
  - amoled 1
  - мин 1

Отзыв 5:
  - цен 1
  - уровн 1
  - тысяч 1
  - смартфон 1
  - рабоч 1
  - отличн 1
  - нужн 1
  - лошадк 1
  - подожда 1
  - ал 1

Отзыв 6:
  - цен 2
  - норм 2
  - стал 1
  - стильн 1
  - неадекватн 1
  - тормоз 1
  - камер 1
  - выгляд 1

Отзыв 7:
  - чехол 2
  - камер 2
  - комплект 2
  - широкоугольн 1
  - шустр 1
  - экран 1
  - сво 1
  - работа 1
  - симк 1
  - используетсясд 1

Отзыв 8:
  - нов 3
  - раньш 2
  - категор 2
  - функц 1
  - теле

Выводы: BoW учитывал только самые частотные слова: "батаре", "особен", "постоя", "телефон". TF-IDF учитывал не только частоту слов, но также и его редкость для данного сбора отзывов: "цен", "норм", "тормоз", "использован", что более информативно для анализа. Косинусная близость низкая (в основном <0.3). Это означает, что отзывы не похожи по содержанию.

**Задание 2: Морфологическая разметка текста**

1. Возьмите отрывок текста (минимум 300 слов) на русском и английском языке.
2. Для русского языка используйте PyMorphy2 или PyMorphy3, для английского - NLTK или SpaCy для проведения морфологического анализа.
3. Выполните следующие операции:
* Определите части речи для каждого слова в тексте
* Для существительных определите падеж, род и число
* Для глаголов определите время, лицо и число
* Создайте частотный словарь частей речи в тексте
4. Разработайте функцию, которая будет автоматически изменять текст, заменяя все существительные на их форму множественного числа (где возможно).
5. Результаты морфологического анализа должны быть представлены в виде таблицы.
6. Оцените и прокомментируйте точность определения морфологических характеристик

In [16]:
# ваш код
russian_text = """Языковой конфликт — это социальный конфликт, тем или иным способом связанный с языковой политикой государства. Языковой конфликт - столкновение между сообществами людей, в основе которого лежат проблемы, связанные с функционированием языков (Т.В. Жеребило). Причины этого конфликта могут лежать в экономической, политической, социальной, религиозной и т. п. сферах, однако он приобретает конкретную форму выражения в связи с языковым вопросом. Доминирующая в социальном отношении группа, которая пытается ограничить возможности других групп, будет в этом случае рассматриваться именно как языковая группа. Нет единой типологии или классификации конфликтов, мы представим вам Современный подход к классификации языковых конфликтов в российской социолингвистике (межэтнические; внутриэтнические; лингвопрагматические). Межэтнический языковой конфликт – конфликт, возникающий между разными этническими группами. Основой межэтнического конфликта всегда является борьба за распределение сфер функционирования между языками, составляющими языковую ситуацию. Внутриэтнический языковой конфликт - языковой конфликт, возникающий между группами людей – представителей одного этноса, обычно не бывают такими острыми, как межэтнические. Лингвопрагматический языковой конфликт - это языковой конфликт понимается как коммуникативная неудача, которая происходит в силу незнания правил коммуникации или некомпетентности одного из участников коммуникации в выборе кода для передачи или получения информации, который не соответствует ситуации общения, провоцирует неправильную интерпретацию высказывания (появление «иных» смыслов).
В целом можно выделить следующие причины языковых конфликтов: неравенство статуса языков (один язык может быть объявлен государственным или официальным, это может привести к чувству угнетения и дискриминации у носителей языка меньшинств), дискриминация в образовании и трудоустройстве (если образование и трудоустройство доступны только на одном языке, это создает барьеры для тех, кто говорит на других языках), политика ассимиляции или политика глоттофагии (подавление и вытеснение языка меньшинства языком доминирующей группы, переход от многоязычия к одноязычию (переход на другой язык)), национализм и идентичность (язык часто является важной частью национальной и культурной идентичности. Угроза языку может восприниматься как угроза идентичности), политические манипуляции (политики могут использовать языковые вопросы для мобилизации поддержки и разжигания конфликтов).
При анализе конкретного языкового конфликта в социолингвистике чаще всего затрагиваются такие темы, как многоязычие, языки меньшинств, социолингвистические результаты миграционных процессов, передача языка от поколения к поколению, переход на другой язык, усвоение языка, соотношение этнических и языковых границ."""
english_text ="""Once upon a time, Fox and Rabbit lived in the dark forest and were best friends. But how could it be?
It was winter when they knew about each other the first time. Due to cold season, it was hard for both of them to find some food. Fox was on his hunt when he saw Rabbit, but the small skinny animal looked so unappetizing and forceless that he decided to find something more nourishing and ran far from Rabbit. And soon he discovered a dead body. It was an eagle that had been shot by a hunter. That was their first meeting that didn’t give them anything.
Six month ago, Fox was running for a mouse, but it was so fast that there was no chance to catch it. So Fox lost it and Rabbit who was growing carrots could see this scene clearly. He didn’t know what he thought about maybe it was a grateful flashback about their winter meeting, but he said:
“Hey, how are you not dead yet? With such disgusting skill in hunting you are absolutely hopeless.”
Fox stopped and looked around as soon as he saw Rabbit, he couldn’t believe that it was real. When his first shock passed, Fox answered:
“What did you say? It’s obvious to me that you don’t have a sense of self-preservation. I got it: you wanna die! ”
“Of cause no, silly fox! You didn’t get anything, I just want to help you. You spared me last winter and I have to pay you back. I do not like to be in debt. So I can lend you a hand but are you ready to get it? ” asked Rabbit. Although his voice was steady, his shaking paw betrayed his nervousness.
Fox could not answer immediately it was so unexpectable that looked like a crazy dream. """

In [17]:
!pip install nltk spacy
!pip install inflect
!pip install pymorphy3

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk
import string
import pymorphy3
import inflect
nltk.download('punkt')
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger_eng')

import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 73.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [18]:
import re

morph = pymorphy3.MorphAnalyzer()
text_tokens_rus = re.findall(r'\b[а-яА-ЯёЁ]+\b', russian_text.lower())
rus_list_of_tagged = []
pos_counts = Counter()

for word in text_tokens_rus:
    parses = morph.parse(word)
    if not parses or parses[0].tag.POS is None:
        continue
    best_parse = parses[0]
    pos = best_parse.tag.POS
    _case = best_parse.tag.case
    gender = best_parse.tag.gender
    number = best_parse.tag.number
    person = best_parse.tag.person
    tense = best_parse.tag.tense
    if pos == 'NOUN':
        rus_list_of_tagged.append(f'{word}_{pos}_{_case}_{gender}_{number}')
    elif pos == 'VERB' or pos == 'INFN' or pos == 'GRND' or pos == 'PRTF' or pos == 'PRTS':
        rus_list_of_tagged.append(f'{word}_{pos}_{tense}_{person}_{number}')
    else:
        rus_list_of_tagged.append(f'{word}_{pos}')
    pos_counts[pos] += 1

print(rus_list_of_tagged)
print("Частотный словарь частей речи в  русском тексте")
for pos, count in pos_counts.most_common():
    print(f"{pos}: {count}")

['языковой_ADJF', 'конфликт_NOUN_nomn_masc_sing', 'это_PRCL', 'социальный_ADJF', 'конфликт_NOUN_nomn_masc_sing', 'тем_CONJ', 'или_CONJ', 'иным_ADJF', 'способом_NOUN_ablt_masc_sing', 'связанный_ADJF', 'с_PREP', 'языковой_ADJF', 'политикой_NOUN_ablt_femn_sing', 'государства_NOUN_gent_neut_sing', 'языковой_ADJF', 'конфликт_NOUN_nomn_masc_sing', 'столкновение_NOUN_accs_neut_sing', 'между_PREP', 'сообществами_NOUN_ablt_neut_plur', 'людей_NOUN_gent_masc_plur', 'в_PREP', 'основе_NOUN_loct_femn_sing', 'которого_ADJF', 'лежат_VERB_pres_3per_plur', 'проблемы_NOUN_nomn_femn_plur', 'связанные_ADJF', 'с_PREP', 'функционированием_NOUN_ablt_neut_sing', 'языков_NOUN_gent_masc_plur', 'т_ADVB', 'в_PREP', 'жеребило_VERB_past_None_sing', 'причины_NOUN_gent_femn_sing', 'этого_NPRO', 'конфликта_NOUN_gent_masc_sing', 'могут_VERB_pres_3per_plur', 'лежать_INFN_None_None_None', 'в_PREP', 'экономической_ADJF', 'политической_ADJF', 'социальной_ADJF', 'религиозной_ADJF', 'и_CONJ', 'т_ADVB', 'п_NOUN_nomn_femn_sing'

In [19]:
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [20]:
text_tokens_eng = word_tokenize(english_text)
tagged = nltk.pos_tag(text_tokens_eng)
list_of_tagged = []

for elem in tagged:
    tag_tog = '_'.join(elem)
    list_of_tagged.append(tag_tog)
print(list_of_tagged)

text_tokens_eng = [t for t in text_tokens_eng if re.search(r"[A-Za-z0-9]", t)]
tagged = pos_tag(text_tokens_eng)
print("Частотный словарь частей речи в  английском тексте")
pos_counts_eng = Counter(tag for _, tag in tagged)
for pos, count in pos_counts_eng.most_common():
    print(f"{pos}: {count}")

['Once_RB', 'upon_IN', 'a_DT', 'time_NN', ',_,', 'Fox_NNP', 'and_CC', 'Rabbit_NNP', 'lived_VBD', 'in_IN', 'the_DT', 'dark_JJ', 'forest_NN', 'and_CC', 'were_VBD', 'best_RBS', 'friends_NNS', '._.', 'But_CC', 'how_WRB', 'could_MD', 'it_PRP', 'be_VB', '?_.', 'It_PRP', 'was_VBD', 'winter_NN', 'when_WRB', 'they_PRP', 'knew_VBD', 'about_IN', 'each_DT', 'other_JJ', 'the_DT', 'first_JJ', 'time_NN', '._.', 'Due_JJ', 'to_TO', 'cold_VB', 'season_NN', ',_,', 'it_PRP', 'was_VBD', 'hard_JJ', 'for_IN', 'both_DT', 'of_IN', 'them_PRP', 'to_TO', 'find_VB', 'some_DT', 'food_NN', '._.', 'Fox_NNP', 'was_VBD', 'on_IN', 'his_PRP$', 'hunt_NN', 'when_WRB', 'he_PRP', 'saw_VBD', 'Rabbit_NNP', ',_,', 'but_CC', 'the_DT', 'small_JJ', 'skinny_NN', 'animal_NN', 'looked_VBD', 'so_RB', 'unappetizing_JJ', 'and_CC', 'forceless_NN', 'that_IN', 'he_PRP', 'decided_VBD', 'to_TO', 'find_VB', 'something_NN', 'more_JJR', 'nourishing_JJ', 'and_CC', 'ran_VBD', 'far_RB', 'from_IN', 'Rabbit_NNP', '._.', 'And_CC', 'soon_RB', 'he_PRP'

In [21]:
def pluralize_russian_nouns(text):
    tokens = re.findall(r'(\b[а-яА-ЯёЁ]+\b|[^\s])', text)
    pluralized_words = []
    for token in tokens:
        if re.match(r'\b[а-яА-ЯёЁ]+\b', token):
            parsed_word = morph.parse(token.lower())[0]
            if 'NOUN' in parsed_word.tag:
                plural_form = parsed_word.inflect({'plur', parsed_word.tag.case})
                if plural_form:
                    if token[0].isupper():
                        pluralized_words.append(plural_form.word.capitalize())
                    else:
                        pluralized_words.append(plural_form.word)
                else:
                    pluralized_words.append(token)
            else:
                pluralized_words.append(token)
        else:
            pluralized_words.append(token)
    reconstructed_text = []
    for i, word in enumerate(pluralized_words):
        reconstructed_text.append(word)
        if i < len(pluralized_words) - 1 and (re.match(r'\b[а-яА-ЯёЁ]+\b', word) and re.match(r'\b[а-яА-ЯёЁ]+\b', pluralized_words[i+1])):
            reconstructed_text.append(' ')

    return "".join(reconstructed_text)

pluralized_russian_text = pluralize_russian_nouns(russian_text)
print(pluralized_russian_text)

Языковой конфликты—это социальный конфликты,тем или иным способами связанный с языковой политиками государств.Языковой конфликты-столкновения между сообществами людей,в основах которого лежат проблемы,связанные с функционированиями языков(Т.В.Жеребило).Причин этого конфликтов могут лежать в экономической,политической,социальной,религиозной и т.п.сферах,однако он приобретает конкретную формы выражений в связей с языковым вопросами.Доминирующая в социальном отношениях группы,которая пытается ограничить возможностей других групп,будет в этом случаях рассматриваться именно как языковая группы.Нет единой типологий или классификаций конфликтов,мы представим вам Современный подходы к классификаций языковых конфликтов в российской социолингвистикам(межэтнические;внутриэтнические;лингвопрагматические).Межэтнический языковой конфликты–конфликты,возникающий между разными этническими группами.Основами межэтнического конфликтов всегда является борьба за распределения сфер функционирований между язы

In [22]:
p = inflect.engine()
nlp = spacy.load("en_core_web_sm")

def pluralize_nouns_spacy(text: str) -> str:
    doc = nlp(text)
    out = []
    for tok in doc:
        if tok.tag_ in ("NN", "NNP"):
            pl = p.plural_noun(tok.text) or p.plural(tok.text)
            out.append(pl + tok.whitespace_)
        else:
            out.append(tok.text_with_ws)

    return "".join(out)

print(pluralize_nouns_spacy(english_text))

eng_plural = pluralize_nouns_spacy(english_text)

Once upon a times, Foxes and Rabbits lived in the dark forests and were best friends. But how could it be?
It was winters when they knew about each other the first times. Due to cold seasons, it was hard for both of them to find some foods. Foxes was on his hunts when he saw Rabbits, but the small skinny animals looked so unappetizings and forcelesses that he decided to find somethings more nourishing and ran far from Rabbits. And soon he discovered a dead bodies. It was an eagles that had been shot by a hunters. That was their first meetings that didn’t give them anythings. 
Six months ago, Foxes was running for a mice, but it was so fast that there was no chances to catch it. So Foxes lost it and Rabbits who was growing carrots could see this scenes clearly. He didn’t know what he thought about maybe it was a grateful flashbacks about their winters meetings, but he said:
“Hey, how are you not dead yet? With such disgusting skills in hunting you are absolutely hopeless.”
Foxes stopped

In [23]:
import pandas as pd
from collections import Counter

rus_map = {
    'NOUN': 'NOUN', 'VERB': 'VERB', 'INFN': 'VERB', 'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADJ',
    'ADVB': 'ADV', 'PRCL': 'PART', 'INTJ': 'INTJ', 'PREP': 'ADP', 'CONJ': 'CONJ', 'NPRO': 'PRON',
    'NUMR': 'NUM', 'PRED': 'OTHER', None: 'OTHER',
    'GRND': 'VERB',
    'PRTF': 'ADJ',
    'ADVB': 'ADV'
}

eng_map = {
    'NN': 'NOUN', 'NNS': 'NOUN', 'NNP': 'NOUN', 'NNPS': 'NOUN',
    'VB': 'VERB', 'VBD': 'VERB', 'VBG': 'VERB', 'VBN': 'VERB', 'VBP': 'VERB', 'VBZ': 'VERB', 'MD': 'VERB',
    'JJ': 'ADJ', 'JJR': 'ADJ', 'JJS': 'ADJ',
    'RB': 'ADV', 'RBR': 'ADV', 'RBS': 'ADV', 'WRB': 'ADV', 'RP': 'ADV',
    'PRP': 'PRON', 'PRP$': 'PRON', 'WP': 'PRON', 'WP$': 'PRON',
    'IN': 'ADP', 'TO': 'ADP',
    'DT': 'DET', 'PDT': 'DET',
    'CC': 'CONJ',
    'CD': 'NUM',
    'UH': 'INTJ',
    '.': 'PUNCT', ',': 'PUNCT', ':': 'PUNCT', '(': 'PUNCT', ')': 'PUNCT', "''": 'PUNCT', '``': 'PUNCT',
    'FW': 'OTHER', 'EX': 'OTHER', 'LS': 'OTHER', 'SYM': 'OTHER',
    'POS': 'PART'
}

normalized_russian_pos_freq = Counter()
for token in rus_list_of_tagged:
    parts = token.split('_')
    pos_tag = parts[1] if len(parts) > 1 else 'OTHER'
    normalized_pos = rus_map.get(pos_tag, 'OTHER')
    normalized_russian_pos_freq[normalized_pos] += 1

normalized_english_pos_freq = Counter()
for elem in list_of_tagged:
    if '_' in elem:
        word, pos_tag = elem.split('_', 1)
        normalized_pos = eng_map.get(pos_tag, 'OTHER')
        normalized_english_pos_freq[normalized_pos] += 1
    else:
        normalized_english_pos_freq['OTHER'] += 1

df_rus = pd.DataFrame.from_dict(normalized_russian_pos_freq, orient='index', columns=['Русский текст'])
df_eng = pd.DataFrame.from_dict(normalized_english_pos_freq, orient='index', columns=['Английский текст'])

df_morph_comparison = pd.concat([df_rus, df_eng], axis=1).fillna(0).astype(int)

df_morph_comparison['Total Count'] = df_morph_comparison['Русский текст'] + df_morph_comparison['Английский текст']
df_morph_comparison = df_morph_comparison.sort_values(by='Total Count', ascending=False).drop(columns='Total Count')

print("Таблица сравнения частот частей речи:")
print("="*50)
print(df_morph_comparison)
print("\n" + "="*50)

print(f"\nВсего слов в русском тексте: {sum(normalized_russian_pos_freq.values())}")
print(f"Всего слов в английском тексте: {sum(normalized_english_pos_freq.values())}")
print(f"Уникальных категорий частей речи: {len(df_morph_comparison)}")

Таблица сравнения частот частей речи:
       Русский текст  Английский текст
NOUN             143                62
ADJ               78                31
VERB              28                80
ADP               36                35
PRON               6                52
PUNCT              0                39
CONJ              23                13
ADV                6                28
DET                0                19
PART               7                 0
OTHER              3                 3
NUM                0                 1


Всего слов в русском тексте: 330
Всего слов в английском тексте: 363
Уникальных категорий частей речи: 12


Выводы: PyMorph3 предоставляет детальную морфологическую информацию, что является его главным преимуществом для более глубокого анализа. Данная библиотека хорошо справилась с разметкой русской морфологии, поскольку в её основе лежат содержательные словари. NLTK довольно точно определяет базовые части речи, что достаточно для большинства прикладных задач. Проблемы могут возникать со словами, не входящими в словарь, или с заимствованиями.

**Задание 3: Синтаксический анализ предложений**

1. Возьмите 2 простых и 3 сложных предложений на русском и английском языке (всего 10 предложений).
2. Используйте SpaCy для построения синтаксических деревьев этих предложений.
3. Для каждого предложения:
* Визуализируйте синтаксическое дерево
* Выделите все подлежащие и сказуемые
* Найдите все пары слов, связанные отношением определения (прилагательное-существительное)
4. Разработайте функцию для извлечения всех объектных и субъектных отношений из предложения в формате (субъект, предикат, объект).
5. Объясните, какие трудности возникают при синтаксическом анализе сложных предложений

In [24]:
!python -m spacy download ru_core_news_md
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 11.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 17.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [25]:
# ваш код
russian_sentences = [
    "Я смотрю во двор.",
    "Я ненавижу грибной дождь.",
    "Каждый раз, когда я тебя вижу, то не узнаю.",
    "Если ты не хочешь идти вместе, просто так и скажи.",
    "Я хотел прибраться до твоего прихода, но не успел."
]

english_sentences = [
    "I heard a loud crack.",
    "She feeds stray dogs every day.",
    "Even when I'm sad, you always find a way to make me laugh.",
    "If I hadn't helped you, you would never have become so rich and successful.",
    "I would have arrived on time, but because of the weather there were terrible traffic jams on the roads."
]

In [26]:
nlp_ru = spacy.load("ru_core_news_md")
nlp_en = spacy.load("en_core_web_md")
from spacy import displacy

In [27]:
print('Визуализация синтакисческого дерева для русского языка\n')
for sentence in russian_sentences:
    doc_ru = nlp_ru(sentence)

    print(f"Предложение {russian_sentences.index(sentence) + 1}")
    displacy.render(doc_ru, style="dep", jupyter=True, options={"distance": 100, "compact": True})

Визуализация дерева зависимостей для русского языка

Предложение 1


Предложение 2


Предложение 3


Предложение 4


Предложение 5


In [28]:
for sentence in english_sentences:
    doc_en = nlp_en(sentence)
    print(f"Sentence {english_sentences.index(sentence) + 1}")
    displacy.render(doc_en, style="dep", jupyter=True, options={"distance": 100, "compact": True})

Sentence 1


Sentence 2


Sentence 3


Sentence 4


Sentence 5


In [35]:
def find_subj_pred_rus(doc):
    root = None
    pred_tokens = []

    for token in doc:
        if token.dep_ == "ROOT":
            root = token
            pred_tokens.append(token)
            break

    if not root:
        return [], []

    changed = True
    while changed:
        changed = False
        for t in doc:
            if t.dep_ == "conj" and t.head in pred_tokens and t.pos_ in {"VERB", "AUX"}:
                if t not in pred_tokens:
                    pred_tokens.append(t)
                    changed = True

    def pred_text(p):
        parts = {p}
        for ch in p.children:
            if ch.dep_ in {"aux", "auxpass", "cop", "neg"}:
                parts.add(ch)
            if ch.dep_ == "xcomp" and ch.pos_ in {"VERB", "AUX"}:
                parts.add(ch)
        return " ".join(tok.text for tok in sorted(parts, key=lambda x: x.i))

    def subjects(p):
        subs = [ch for ch in p.children if ch.dep_ in {"nsubj", "nsubjpass", "nsubj:pass"}]
        if subs:
            return subs
        if p.dep_ == "conj":
            return subjects(p.head)
        return []

    all_subj = []
    all_pred = []
    for p in sorted(pred_tokens, key=lambda x: x.i):
        all_pred.append(pred_text(p))
        all_subj.extend([s.text for s in subjects(p)])

    return all_subj, all_pred
for text in russian_sentences:
  doc = nlp_ru(text)
  subject, predicate = find_subj_pred_rus(doc)
  print(f"Предложение: {text}")
  print(f"Подлежащее: {subject}, Сказуемое: {predicate}")
  print()

Предложение: Я смотрю во двор.
Подлежащее: ['Я'], Сказуемое: ['смотрю']

Предложение: Я ненавижу грибной дождь.
Подлежащее: ['Я'], Сказуемое: ['ненавижу']

Предложение: Каждый раз, когда я тебя вижу, то не узнаю.
Подлежащее: [], Сказуемое: ['узнаю']

Предложение: Если ты не хочешь идти вместе, просто так и скажи.
Подлежащее: [], Сказуемое: ['скажи']

Предложение: Я хотел прибраться до твоего прихода, но не успел.
Подлежащее: ['Я', 'Я'], Сказуемое: ['хотел прибраться', 'успел']



In [36]:
def find_subj_pred_eng(doc):
    subj = []
    pred_tokens = []
    root = None
    for token in doc:
        if token.dep_ == "ROOT":
            root = token
            pred_tokens.append(token)
            break
    if not root:
        return [], []

    for token in doc:
        if token.dep_ in ["aux", "auxpass"] and token.head in [root] + [t for t in pred_tokens]:
            pred_tokens.append(token)
    pred = [doc[i].text for i in sorted([t.i for t in pred_tokens])]
    for token in doc:
        if token.dep_ in ["nsubj", "nsubjpass"]:
            current = token
            while current.head != current:
                if current.head in pred_tokens or current.head == root:
                    subj.append(token.text)
                    break
                current = current.head
    if not subj:
        for token in doc:
            if token.dep_ in ["nsubj", "nsubj:pass"]:
                subj.append(token.text)
    return subj, pred
for text in english_sentences:
    doc = nlp_en(text)
    subject, predicate = find_subj_pred_eng(doc)
    print(f"Предложение: {text}")
    print(f"Подлежащее: {subject}, Сказуемое: {predicate}")
    print()

Предложение: I heard a loud crack.
Подлежащее: ['I'], Сказуемое: ['heard']

Предложение: She feeds stray dogs every day.
Подлежащее: ['She'], Сказуемое: ['feeds']

Предложение: Even when I'm sad, you always find a way to make me laugh.
Подлежащее: ['I', 'you', 'me'], Сказуемое: ['find']

Предложение: If I hadn't helped you, you would never have become so rich and successful.
Подлежащее: ['I', 'you'], Сказуемое: ['would', 'have', 'become']

Предложение: I would have arrived on time, but because of the weather there were terrible traffic jams on the roads.
Подлежащее: ['I'], Сказуемое: ['would', 'have', 'arrived']



In [41]:
print("Отношение прилагательное-существительное в русских предложениях:")
for sentence in russian_sentences:
    doc_ru = nlp_ru(sentence)
    for token in doc_ru:
        if token.dep_ == "amod":
            print(f"Объект '{token.text}' зависит от существительного '{token.head.text}'")

print("\nОтношение прилагательное-существительное в английских предложениях:")
for sentence in english_sentences:
    doc_en = nlp_en(sentence)
    for token in doc_en:
        if token.dep_ == "amod":
            print(f"Объект '{token.text}' зависит от существительного '{token.head.text}'")


Отношение прилагательное-существительное в русских предложениях:
Объект 'грибной' зависит от существительного 'дождь'

Отношение прилагательное-существительное в английских предложениях:
Объект 'loud' зависит от существительного 'crack'
Объект 'stray' зависит от существительного 'dogs'
Объект 'terrible' зависит от существительного 'jams'


In [42]:
def find_SVO(sentence, lang='en'):
    if lang == 'rus':
        doc = nlp_ru(sentence)
        for token in doc:
            if token.dep_ == "nsubj":
                subj = token.text
            if token.dep_ == "obj":
                obj = token.text
                predicate = token.head.text
                print(f"{subj} {predicate} {obj} ")
    else:
        doc = nlp_en(sentence)
        for token in doc:
            if token.dep_ in {"nsubj", "nsubjpass", "csubj", "csubjpass"}:
                subj = token.text
                predicate = token.head.text
            if token.dep_ in {"dobj", "iobj", "obj"}:
                obj = token.text
                print(f"{subj} {predicate} {obj}")

print("Примеры объектных и субъектных отношений из русских предложений:")
for sentence in russian_sentences:
    find_SVO(sentence, 'rus')
print("\nПримеры объектных и субъектных отношений из английских предложений:")
for sentence in english_sentences:
    find_SVO(sentence, 'en')

Примеры объектных и субъектных отношений из русских предложений:
Я ненавижу дождь 
я вижу тебя 

Примеры объектных и субъектных отношений из английских предложений:
I heard crack
She feeds dogs
you find way
I helped you


Анализ трудностей: 1)наличие придаточных предложений (сложенная структура); 2)свободный порядок слов в русском языке; 3)несколько сказуемых у одного подлежащего, что вызывает затруднения с поиском.

**Задание 4: Распознавание именованных сущностей (Named Entity Recognition)**

1. Подготовьте корпус из 10 новостных текстов, содержащий различные типы именованных сущностей (имена людей, организации, географические названия, даты и т.д.) на английском или русском языке.
2. Используйте SpaCy для автоматического распознавания именованных сущностей.
3. Реализуйте свой простой метод для распознавания имен людей и географических названий с помощью регулярных выражений и словарей.
4. Сравните результаты работы SpaCy и вашего метода:
* Рассчитайте точность (precision), полноту (recall) и F1-меру для вашего метода относительно результатов SpaCy
* Проанализируйте ошибки обоих подходов, какие типы ошибок характерны для каждого подхода
5. Представьте сравнение результатов в виде таблицы

In [43]:
# ваш код
documents = [
    "В пятницу, 23 января, приморский баскетбольный клуб «Динамо» в рамках домашней серии Суперлиги примет «Тамбов». Матч на площадке «Олимпийца» начнётся в 19:00. Приобрести билет можно на сайте «Афиша» на VL.ru. Потерпев поражение на старте домашней серии от ЦСКА-2, команда из Владивостока готовится реабилитироваться перед своими болельщиками уже в ближайшем матче. В гости в приморскую столицу прибыл один из аутсайдеров чемпионата Суперлиги – «Тамбов». В декабре команды уже пересекались на паркете «Олимпийца», но в рамках Кубка России, и тогда сильнее оказались приморские баскетболисты. По итогам первого круга чемпионата «Тамбов» расположился на 14-м месте, а в новый год ворвался поражением (70:75) от «Металлурга», в который перешёл Михайло Богданович. Реабилитироваться тамбовчанам удалось в своей последней игре – в рамках «волчьего» дерби коллектив Олега Игумнова победил лидера чемпионата ЧБК в овертайме – 101:87. Лучшим на паркете стал Владимир Заболотнев, набравший свои рекордные 23 очка в сезоне. С 4 победами и 11 поражениями «Тамбов» занимает 13-е место в турнирной таблице, в среднем набирая 73 очка за матч. Самый результативный игрок «Тамбова» – Артём Орехов (11,5 очка). В тройке сильнейших – Даниил Синегубов (10,9 очка) и Филипп Стойко (9,9 очка). По реализации двухочковых бросков (55%) и на подборах (5,9 пд) лучший Дмитрий Карбусов, на трёхочковой дистанции главную угрозу составляет Филипп Стойко (38,5%). По голевым передачам лидирует Владимир Заболотнев (4,4 гп)."
    "В пятницу, 23 января, во Фрунзенском суде собралось более 30 юристов и около 20 полицейских, которые пришли поддержать своих коллег. У члена Приморской краевой коллегии адвокатов и женщины-следователя месяц назад произошёл конфликт, который вылился в уголовное дело о применении насилия в отношении представителя власти. Адвоката отправили под домашний арест до середины марта – все дела, которые он ведёт, придётся передать другим защитникам. Руслан Омельченко – адвокат Приморской краевой коллегии «Щит и меч». Несколько лет назад он защищал в суде бывшего главу Владивостока Игоря Пушкарёва. Сейчас является адвокатом одного из фигурантов дела о хищении акций «Восточной верфи» и ведёт несколько тяжких и особо тяжких дел."
    "Реконструировать старое, обветшавшее здание кинотеатра – такую задачу поставил глава Арсеньевского городского округа А.А. Дронин, учитывая многочисленные просьбы жителей города разных возрастов. Благодаря усилиям заместителя председателя Думы городского округа В.А. Авагимяна найден надежный инвестор, много лет работающий в области кинопроката. В 2013 году администрация города подписала концессионное соглашение на здание кинотеатра с ООО «Новая Волна». Перед инвестором встала задача – полностью реконструировать здание, установить новое оборудование, создать уютную атмосферу для посетителей. Сегодня выполнена большая часть внутренних работ: светлым и просторным стал холл здания, на полу уже выложена плитка, оформлен потолок, выполнены ремонтные работы в подсобных помещениях. В зрительных залах идет монтаж экранов, скоро здесь будут установлены комфортные кресла. Уже оформлен потолок – на темном фоне мерцают софиты. В обновленном кинотеатре предусмотрены два зрительных зала – на 260 и 120 посадочных мест. В них, кроме просмотра кинолент, можно будет проводить праздники, конференции, творческие встречи и другие мероприятия. Готова к установке современная киноаппаратура, которая обеспечит высокое качество кинопоказа. "
    "18 июня – 90 лет со дня рождения Юрия Мефодьевича Соломина (1935-2024), исполнителя роли В.К. Арсеньева, Народного артиста СССР, Почётного гражданина города Арсеньева. Юрий Соломин родился в Чите в семье музыкантов. Страстная мечта стать актёром привела его в Москву. В 1953 году поступил в Высшее театральное училище им. М.С. Щепкина. С 1957 года Юрий Соломин – в труппе Малого театра, 1988 года – художественный руководитель Малого театра. С 1990 по 1992 год занимал пост Министра культуры РФ. Юрий Мефодьевич Соломин в театре и кино сыграл более 100 ролей. Для нас, арсеньевцев, он навсегда останется талантливым исполнителем роли Владимира Клавдиевича Арсеньева в фильме Акиры Куросавы «Дерсу Узала». Актер так говорил о г. Арсеньеве: «У меня о вашем городе остались самые светлые, добрые, прекрасные воспоминания». В 1975 году за активное участие в пропаганде отечественного киноискусства Ю.М. Соломину присвоено звание «Почётный гражданин города Арсеньева». Информация Централизованной библиотечной системы"
    "Более 5 тысяч жилых домов разрушены из-за наводнений в Мозамбике, повреждены важнейшие объекты инфраструктуры, парализованы ключевые транспортные коридоры, сообщили РИА Новости в российской дипмиссии. 'Из-за наводнений и паводков были разрушены более 5 тысяч жилых домов, пострадали важнейшие объекты инфраструктуры, парализованы многие ключевые транспортные коридоры', - сказали агентству в посольстве."
    "Пакистан заинтересован в расширении торговли и наращивании делового взаимодействия с Евразийским экономический союз (ЕАЭС), сообщили РИА Новости в пресс-службе Евразийской экономической комиссии (ЕЭК). В пятницу министр по торговле Евразийской экономической комиссии Андрей Слепнев провел встречу с чрезвычайным и полномочным послом Исламской Республики Пакистан в РФ Файсалом Ниазом Тирмизи."
    "Визит спикера парламента Армении Алена Симоняна в Россию запланирован на 5-6 февраля, сообщает пресс-служба парламента республики. Отмечается, что детали визита были обсуждены в ходе телефонной беседы Симоняна с с главой Совета Федерации РФ Валентиной Матвиенко, состоявшейся по инициативе российской стороны."
    "Похороны итальянского модельера, основателя модного дома Valentino Валентино Гаравани проходят в римской базилике Санта-Мария-дельи-Анджели-э-дей-Мартири, передает корреспондент РИА Новости. Гроб с кутюрье у прибывшего к церкви катафалка встретили его семья и близкие, которые зашли вслед за ним внутрь здания. В базилике тело усопшего сопровождали траурные ноты 'Лакримозы' Моцарта. Помимо окружения Валентино проститься с ним пришли его сотрудники и представители мира моды: среди них оказались Том Форд, Донателла Версаче, Наталья Водянова и Анна Винтур, а также актрисы Энн Хэтэуэй и Элизабет Херли."
    "Хакеры похитили данные более 17 миллионов аккаунтов в соцсети Instagram*, сообщила специализирующаяся на кибербезопасности американская компания Malwarebytes.'Киберпреступники похитили чувствительную информацию 17,5 миллиона аккаунтов в Instagram*, включая имена пользователей, физические адреса, номера телефонов, электронные адреса и более', - написала компания на своей странице в соцсети Х. Портал Notebookcheck сообщает, что данные могут быть использованы для фишинговых атак и воровства аккаунтов. По этой информации, уязвимость, из-за которой произошла утечка, возникла в 2024 году."
    "Присутствие в Совете мира стран, поддерживающих Палестину, может привести к образованию клубка разных проблем на региональном уровне, заявил НСН бывший посол Израиля в России и на Украине Цви Маген. Большинство стран ЕС отклонили приглашение США войти в Совет мира по Газе, пишет Financial Times. По версии издания, это произошло из-за того, что в структуру был приглашен президент России Владимир Путин. Премьер Израиля согласился войти в Совет мира, несмотря на критику состава исполнительного комитета: в него включили Турцию, которую израильские власти рассматривают как ключевого регионального соперника. Цви Маген отметил, что израильскую сторону совершенно не смущает присутствие Путина в Совете мира."
]

In [52]:
import spacy
import re
import pandas as pd

nlp = spacy.load("ru_core_news_sm")

In [58]:
def extract_spacy_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def simple_ner(text):
    entities = []
    known_names = [
        'Михайло Богданович', 'Владимир Заболотнев', 'Олег Игумнов',
        'Артём Орехов', 'Даниил Синегубов', 'Филипп Стойко',
        'Дмитрий Карбусов', 'Руслан Омельченко', 'Игорь Пушкарёв',
        'А.А. Дронин', 'В.А. Авагимян', 'Юрий Мефодьевич Соломин',
        'Юрий Соломин', 'Владимир Клавдиевич Арсеньев', 'Акира Куросава',
        'Мозамбик', 'Андрей Слепнев', 'Файсал Ниаз Тирмизи',
        'Ален Симонян', 'Валентина Матвиенко', 'Валентино Гаравани',
        'Том Форд', 'Донателла Версаче', 'Наталья Водянова',
        'Анна Винтур', 'Энн Хэтэуэй', 'Элизабет Херли', 'Цви Маген',
        'Владимир Путин'
    ]
    known_locations = [
        'Владивосток', 'Тамбов', 'Россия', 'Приморский край',
        'Фрунзенский суд', 'Арсеньев', 'Арсеньевский городской округ',
        'Чита', 'Москва', 'Мозамбик', 'Пакистан', 'ЕАЭС', 'ЕЭК',
        'Армения', 'Рим', 'США', 'Израиль', 'Украина', 'Турция', 'ЕС'
    ]

    name_patterns = [
        r'([А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+)',
        r'([А-ЯЁ]\.\s?[А-ЯЁ]\.\s[А-ЯЁ][а-яё]+)',
        r'([А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+)',
    ]
    location_patterns = [
        r'(город[аеу]?\s[А-ЯЁ][а-яё]+)',
        r'([А-ЯЁ][а-яё]+\sкрай)',
        r'(в\s[А-ЯЁ][а-яё]+)',
        r'(на\s[А-ЯЁ][а-яё]+)',
        r'([А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+\sокруг)',
    ]

    for name in known_names:
        if name in text:
            entities.append((name, 'PERSON'))

    for location in known_locations:
        if location in text:
            entities.append((location, 'LOC'))

    for pattern in name_patterns:
        for match in re.finditer(pattern, text):
            name = match.group()
            if name not in [e[0] for e in entities]:
                entities.append((name, 'PERSON'))

    for pattern in location_patterns:
        for match in re.finditer(pattern, text):
            location = match.group()
            if location not in [e[0] for e in entities]:
                entities.append((location, 'LOC'))

    unique_entities = []
    seen = set()
    for ent in entities:
        if ent[0] not in seen:
            unique_entities.append(ent)
            seen.add(ent[0])

    return unique_entities

def calculate_metrics(predicted, reference):
    predicted_set = set([entity[0].lower() for entity in predicted])
    reference_set = set([entity[0].lower() for entity in reference])

    true_positives = len(predicted_set.intersection(reference_set))
    false_positives = len(predicted_set - reference_set)
    false_negatives = len(reference_set - predicted_set)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1, true_positives, false_positives, false_negatives

spacy_results = []
simple_results = []
metrics = []

print("\nСущности, которые распознал каждый метод:\n")

for i, document in enumerate(documents):
    print(f"\n{'='*80}")
    print(f"Документ {i+1}:")
    print(f"{document[:150]}..." if len(document) > 150 else document)

    spacy_entities = extract_spacy_entities(document)
    spacy_results.append(spacy_entities)

    simple_entities = simple_ner(document)
    simple_results.append(simple_entities)

    print(f"\nSpaCy ({len(spacy_entities)} сущностей):")
    for entity, label in spacy_entities:
        print(f"  {entity:40} [{label}]")

    print(f"\nПростой метод ({len(simple_entities)} сущностей):")
    for entity, label in simple_entities:
        print(f"  {entity:40} [{label}]")

    precision, recall, f1, tp, fp, fn = calculate_metrics(simple_entities, spacy_entities)
    metrics.append({
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': tp,
        'fp': fp,
        'fn': fn
    })

    print(f"\nМетрики для документа {i+1}:")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall: {recall:.3f}")
    print(f"  F1: {f1:.3f}")
    print(f"  Совпадения (TP): {tp}")
    print(f"  Ложные срабатывания (FP): {fp}")
    print(f"  Пропущенные (FN): {fn}")

print("\n" + "="*80)
print("Общая таблица полученных результатов:\n")

table_data = []
for i in range(len(documents)):
    table_data.append({
        'Документ': i+1,
        'Precision': f"{metrics[i]['precision']:.3f}",
        'Recall': f"{metrics[i]['recall']:.3f}",
        'F1': f"{metrics[i]['f1']:.3f}",
        'TP': metrics[i]['tp'],
        'FP': metrics[i]['fp'],
        'FN': metrics[i]['fn'],
        'Сущностей (spaCy)': len(spacy_results[i]),
        'Сущностей (простой)': len(simple_results[i])
    })

avg_precision = sum(m['precision'] for m in metrics) / len(metrics)
avg_recall = sum(m['recall'] for m in metrics) / len(metrics)
avg_f1 = sum(m['f1'] for m in metrics) / len(metrics)
total_tp = sum(m['tp'] for m in metrics)
total_fp = sum(m['fp'] for m in metrics)
total_fn = sum(m['fn'] for m in metrics)
total_spacy = sum(len(r) for r in spacy_results)
total_simple = sum(len(r) for r in simple_results)

table_data.append({
    'Документ': 'Среднее/Итого',
    'Precision': f"{avg_precision:.3f}",
    'Recall': f"{avg_recall:.3f}",
    'F1': f"{avg_f1:.3f}",
    'TP': total_tp,
    'FP': total_fp,
    'FN': total_fn,
    'Сущностей (spaCy)': total_spacy,
    'Сущностей (простой)': total_simple
})

df = pd.DataFrame(table_data)
print(df.to_string(index=False))


Сущности, которые распознал каждый метод:


Документ 1:
В пятницу, 23 января, приморский баскетбольный клуб «Динамо» в рамках домашней серии Суперлиги примет «Тамбов». Матч на площадке «Олимпийца» начнётся ...

SpaCy (109 сущностей):
  Динамо                                   [ORG]
  Олимпийца                                [ORG]
  Афиша                                    [ORG]
  VL.ru                                    [ORG]
  ЦСКА-2                                   [ORG]
  Владивостока                             [LOC]
  Олимпийца                                [ORG]
  России                                   [LOC]
  Металлурга                               [ORG]
  Михайло Богданович                       [PER]
  Олега Игумнова                           [PER]
  ЧБК                                      [ORG]
  Владимир Заболотнев                      [PER]
  Тамбов                                   [ORG]
  Тамбова                                  [ORG]
  Артём Орехов                

Анализ ошибок: SpaCy может не распознавать составные имена ("А.А. Дронин") или не правильно определить типы сущностей. Также, пропускает редко встречающиеся или незнакомые сущности. Просто метод зависит от заранее определённых списков слов и не распознаёт контекстные сущности.